# Maze Solver -  Modal Based Agent with with Percept Memory

In [1]:
import random
import numpy as np

In [2]:
'''
This class is made to either take in a given maze in an array form along with the max limit and goal state or just
randomly make a maze (It does not necessarily have to be a functional maze) which might not have a path too but it useful
to test if our agent and know of it will react and which action it will take
'''

class maze_environment:
    def __init__(self, limit=None, maze=None, goal=None):
        if(maze is None and limit is None):
            self.limit = random.randint(5, 15)
            self.goal = (self.limit-1, self.limit-1)
            self.maze = []
            for i in range(self.limit):
                row = []
                for j in range(self.limit):
                    row.append(random.choice(['F', 'B']))
                self.maze.append(row)
            self.maze[0][0] = 'F'
            self.maze[self.limit-1][self.limit-1] = 'F'
        else:
            self.maze = maze
            self.limit = limit
            self.goal = goal
    
    def get_limit(self):
        return self.limit
    
    def show_maze(self):
        for i in range(self.limit):
            for j in range(self.limit):
                print(self.maze[i][j], end=" ")
            print()
    
    def get_goal(self):
        return self.goal
    
    def get_position(self, x, y):
        if(x > self.limit-1 or y > self.limit-1):
            return 'B'
        if(x < 0 or y < 0):
            return 'B'
        return self.maze[x][y]

### Self Driving agent with Percept Memory

This agent keeps track of the last time it had to take a decision between two moves using a Queue data structure (i.e follow FIFO), this is will be useful if the agent reaches a deadend then it will revert back to that location and continue from there. This gives it the advantage that it does not have to backtrack from the current location instead it can directly teleport back to a better location and continue with its approach. This fixes the problem of getting stuck in loops or not reaching the braching point and thus reduces the time taken drastically. 

In [3]:
class self_driving_agent_memory:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.total_steps = 0
        self.surrondings = None
        self.memory = []
        self.recentlyPoped = []
        self.prev_move = -1
    
    def move(self, env):
        if(self.x == env.get_goal()[0] and self.y == env.get_goal()[1]):
            print("You have solved the maze and reached the end...")
            return False
        elif(self.total_steps >= env.limit*env.limit):
            print("Did not halt, mostly unsolvable maze...")
            return False
        else:
            self.surrondings = [env.get_position(self.x+1, self.y),
                                env.get_position(self.x, self.y+1),
                                env.get_position(self.x-1, self.y), 
                                env.get_position(self.x, self.y-1)]
            possible_moves = []
            num_moves = 0
            for i in range(len(self.surrondings)):
                if self.surrondings[i] == 'F':
                    possible_moves.append(i)
                    num_moves += 1
            
            if(self.prev_move == 0):
                if(2 in possible_moves):
                    possible_moves.remove(2)
            elif(self.prev_move == 2):
                if(0 in possible_moves):
                    possible_moves.remove(0)
            elif(self.prev_move == 1):
                if(3 in possible_moves):
                    possible_moves.remove(3)
            elif(self.prev_move == 3):
                if(1 in possible_moves):
                    possible_moves.remove(1)
                    
            if(num_moves > 1):
                if(not self.recentlyPoped):
                    self.memory.append([self.x, self.y])
                elif(not(self.recentlyPoped[0] == self.x and self.recentlyPoped[1] == self.y)):
                    self.memory.append([self.x, self.y])
            
            if(possible_moves):
                random_move = random.choice(possible_moves)
                if(random_move == 0):
                    self.x += 1
                    self.prev_move = 0
                    print("Go Down")
                elif(random_move == 1):
                    self.y += 1
                    self.prev_move = 1
                    print("Go Right")
                elif(random_move == 2):
                    self.x -= 1
                    self.prev_move = 2
                    print("Go Up")
                elif(random_move == 3):
                    self.y -= 1
                    self.prev_move = 3
                    print("Go Left")
                else:
                    print("Maze is blocked for all the places...")
                    return False
            else:
                if(not self.memory):
                    print("Unsolvable maze, no route to exit...")
                    return False
                last_choice_position = self.memory.pop(0)
                self.x = last_choice_position[0]
                self.y = last_choice_position[1]
                self.recentlyPoped = last_choice_position
                self.total_steps = 0
            
            self.total_steps += 1
            return True

    def tell_pos(self):
        print("I am currently at the position ({}, {})".format(self.x, self.y))

### Running the agent in different environments

In [4]:
#Randomly generated environment
my_env = maze_environment()
my_agent = self_driving_agent_memory()

print("Limit is: ", my_env.get_limit())
print("Goal is: ", my_env.get_goal())
my_env.show_maze()

Limit is:  6
Goal is:  (5, 5)
F B B B F F 
F B F F F F 
B B B B B B 
B F B B B B 
B B F F F B 
B F B B F F 


In [5]:
status = True
while(status):
    status = my_agent.move(my_env)
    my_agent.tell_pos()

Go Down
I am currently at the position (1, 0)
Unsolvable maze, no route to exit...
I am currently at the position (1, 0)


In [6]:
#Custom maze 1 - Single path to goal
cust_maze = [['F', 'B', 'B', 'B', 'F'],
 ['F', 'B', 'B', 'B', 'B'],
 ['F', 'F', 'F', 'B', 'B'],
 ['B', 'B', 'F', 'F', 'F'],
 ['B', 'B', 'B', 'B', 'F']]

In [7]:
my_env2 = maze_environment(limit=5, maze=cust_maze, goal=(4, 4))
my__agent2 = self_driving_agent_memory()

print("Limit is: ", my_env2.get_limit())
print("Goal is: ", my_env2.get_goal())
my_env2.show_maze()

Limit is:  5
Goal is:  (4, 4)
F B B B F 
F B B B B 
F F F B B 
B B F F F 
B B B B F 


In [8]:
status = True
while(status):
    status = my__agent2.move(my_env2)
    my__agent2.tell_pos()

Go Down
I am currently at the position (1, 0)
Go Down
I am currently at the position (2, 0)
Go Right
I am currently at the position (2, 1)
Go Right
I am currently at the position (2, 2)
Go Down
I am currently at the position (3, 2)
Go Right
I am currently at the position (3, 3)
Go Right
I am currently at the position (3, 4)
Go Down
I am currently at the position (4, 4)
You have solved the maze and reached the end...
I am currently at the position (4, 4)


In [9]:
#Custom maze 2 - One terminal branch and double width path 
cust_maze2 = [['F', 'F', 'B', 'B', 'F'],
              ['F', 'F', 'B', 'B', 'B'],
              ['B', 'F', 'F', 'F', 'F'],
              ['B', 'B', 'F', 'B', 'F'],
              ['B', 'B', 'B', 'B', 'F']]

In [10]:
my_env3 = maze_environment(limit=5, maze=cust_maze2, goal=(4, 4))
my__agent3 = self_driving_agent_memory()

print("Limit is: ", my_env3.get_limit())
print("Goal is: ", my_env3.get_goal())
my_env3.show_maze()

Limit is:  5
Goal is:  (4, 4)
F F B B F 
F F B B B 
B F F F F 
B B F B F 
B B B B F 


In [11]:
status = True
while(status):
    status = my__agent3.move(my_env3)
    my__agent3.tell_pos()

Go Right
I am currently at the position (0, 1)
Go Down
I am currently at the position (1, 1)
Go Left
I am currently at the position (1, 0)
Go Up
I am currently at the position (0, 0)
Go Right
I am currently at the position (0, 1)
Go Down
I am currently at the position (1, 1)
Go Left
I am currently at the position (1, 0)
Go Up
I am currently at the position (0, 0)
Go Right
I am currently at the position (0, 1)
Go Down
I am currently at the position (1, 1)
Go Down
I am currently at the position (2, 1)
Go Right
I am currently at the position (2, 2)
Go Right
I am currently at the position (2, 3)
Go Right
I am currently at the position (2, 4)
Go Down
I am currently at the position (3, 4)
Go Down
I am currently at the position (4, 4)
You have solved the maze and reached the end...
I am currently at the position (4, 4)


In [12]:
#Custome maze 3 - Multiple terminal branches and last one leads to goal.
cust_maze3 = [['F', 'F', 'F', 'B', 'B'],
              ['F', 'F', 'B', 'B', 'F'],
              ['F', 'B', 'B', 'B', 'B'],
              ['F', 'B', 'F', 'F', 'F'],
              ['F', 'F', 'F', 'B', 'F']]

In [13]:
my_env4 = maze_environment(limit=5, maze=cust_maze3, goal=(4, 4))
my_agent4 = self_driving_agent_memory()

print("Limit is: ", my_env4.get_limit())
print("Goal is: ", my_env4.get_goal())
my_env4.show_maze()

Limit is:  5
Goal is:  (4, 4)
F F F B B 
F F B B F 
F B B B B 
F B F F F 
F F F B F 


In [14]:
status = True
while(status):
    status = my_agent4.move(my_env4)
    my_agent4.tell_pos()

Go Down
I am currently at the position (1, 0)
Go Right
I am currently at the position (1, 1)
Go Up
I am currently at the position (0, 1)
Go Left
I am currently at the position (0, 0)
Go Down
I am currently at the position (1, 0)
Go Right
I am currently at the position (1, 1)
Go Up
I am currently at the position (0, 1)
Go Left
I am currently at the position (0, 0)
Go Down
I am currently at the position (1, 0)
Go Right
I am currently at the position (1, 1)
Go Up
I am currently at the position (0, 1)
Go Left
I am currently at the position (0, 0)
Go Down
I am currently at the position (1, 0)
Go Down
I am currently at the position (2, 0)
Go Down
I am currently at the position (3, 0)
Go Down
I am currently at the position (4, 0)
Go Right
I am currently at the position (4, 1)
Go Right
I am currently at the position (4, 2)
Go Up
I am currently at the position (3, 2)
Go Right
I am currently at the position (3, 3)
Go Right
I am currently at the position (3, 4)
Go Down
I am currently at the posit

As we can see this is a much better agent which has more consistency present in it along with a better judgement of the environment as it keeps track of the last decision point so it also know if this is a blocked maze (No way to goal) and return it as the result.